In [1]:
import random
import time
import sys
import gc
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model
import pickle
from tqdm import tqdm

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


**re-create rank_dic for the hybrid model**

In [2]:
# df_train_filtered=pd.read_pickle('df_train_filtered.pkl')

# # Mean rating for all movies
# C = df_train_filtered['Rating'].mean()

# # Mean rating for all movies separately
# R = df_train_filtered.groupby(['Movie']).mean()['Rating'].values

# # Rating freqency for all movies separately
# v = df_train_filtered.groupby(['Movie']).count()['Rating'].values

# # Number of minimum votes to be considered
# m = v.min()

# # Weighted formula to compute the weighted rating
# weighted_score = pd.DataFrame(v/(v+m)*R+m/(v+m)*C, columns=['weighted_score'])
# weighted_score.set_index(np.sort(df_train_filtered['Movie'].unique()), inplace=True)
# weighted_score.sort_values(by='weighted_score', ascending=False, inplace=True)

# from collections import OrderedDict

# rank_dic=OrderedDict()

# for i in range(weighted_score.shape[0]):
#     rank_dic[weighted_score.index[i]]=weighted_score.iloc[i,0]

# del df_train_filtered, C, R, v, m, weighted_score

**save the re-created rank_dic for the hybrid model**

In [3]:
# filtered_rank_dic_list=[]
# for key in rank_dic:
#     filtered_rank_dic_list.append([key, rank_dic[key]])
    
# with open('filtered_rank_dic_list.txt', 'wb') as fp:
#     pickle.dump(filtered_rank_dic_list, fp)

**load dictionaries for the hybrid model**

In [4]:
with open('filtered_user_id_mapping_list.txt', 'rb') as fp:
    filtered_user_id_mapping_list=pickle.load(fp)
    
with open('filtered_movie_id_mapping_list.txt', 'rb') as fp:
    filtered_movie_id_mapping_list=pickle.load(fp)
    
with open('filtered_rank_dic_list.txt', 'rb') as fp:
    filtered_rank_dic_list=pickle.load(fp)
    
with open('bert_dic_list.txt', 'rb') as fp:
    bert_dic_list=pickle.load(fp)
    

filtered_user_id_mapping={}

filtered_movie_id_mapping={}

filtered_rank_dic={}

bert_dic={}

for ele in filtered_user_id_mapping_list:
    filtered_user_id_mapping[ele[0]]=ele[1]
    
for ele in filtered_movie_id_mapping_list:
    filtered_movie_id_mapping[ele[0]]=ele[1]
    
for ele in filtered_rank_dic_list:
    filtered_rank_dic[ele[0]]=ele[1]
    
for ele in bert_dic_list:
    bert_dic[ele[0]]=ele[1]
    
del filtered_user_id_mapping_list, filtered_movie_id_mapping_list, filtered_rank_dic_list, bert_dic_list

**load models**

In [5]:
hybrid_model = tf.keras.models.load_model('./hybrid.h5')

2022-03-16 17:48:31.222829: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


**define a function that makes prediction**

In [6]:
def hybrid_pred(user_id, movie_id, filtered_user_id_mapping,\
              filtered_movie_id_mapping, filtered_rank_dic, hybrid_model):

    user=filtered_user_id_mapping[user_id]  
    movie=filtered_movie_id_mapping[movie_id]
    overview=bert_dic[movie_id]
    pred=hybrid_model.predict([np.array([user]), np.array([movie])\
                       ,np.array([overview])\
                       ])[0,0]+filtered_rank_dic[movie_id]
    
    if pred<1: pred=1
    elif pred>5: pred=5
    return pred

**try to make a prediction**

In [7]:
hybrid_users=list(filtered_user_id_mapping.keys())
hybrid_movies=list(filtered_movie_id_mapping.keys())

In [8]:
len(hybrid_users)

150245

In [9]:
user_id=hybrid_users[1130]
movie_id=hybrid_movies[1701]
hybrid_pred(user_id, movie_id, filtered_user_id_mapping, filtered_movie_id_mapping,\
            filtered_rank_dic, hybrid_model),\
filtered_rank_dic[movie_id]

2022-03-16 17:48:32.712143: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


(1.6041499213309378, 2.300797505769262)

In [10]:
layers=hybrid_model.layers
for i in range(len(layers)):
    print(i)
    print(layers[i])
    print(layers[i].trainable)

0
True
1
True
2
True
3
True
4
True
5
True
6
True
7
True
8
True
9
True
10
True
11
True
12
True
13
True
14
True
15
True


In [11]:
hybrid_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
movie (InputLayer)              [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 128)       19244160    user[0][0]                       
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 128)       542592      movie[0][0]                      
____________________________________________________________________________________________

**freeze all the dense layers and the movie embedding layer**

In [12]:
layers=hybrid_model.layers
layers[3].trainable=False
layers[9].trainable=False
layers[13].trainable=False
layers[14].trainable=False
layers[15].trainable=False

**try make a movie recommendation for a user that hasn't rated any movie yet**

In [13]:
user=len(hybrid_users)+0

aaa=list(filtered_rank_dic.keys())

# map movie ids to movie vocabulary number
X_bert=np.array(pd.Series(np.array(aaa)).map(bert_dic).tolist())
# map user ids to user vocabulary number
X_user=np.array([user for i in range(X_bert.shape[0])])

Y=hybrid_model.predict([X_user, pd.Series(aaa).map(filtered_movie_id_mapping).values\
                 ,X_bert\
                ])
Y=Y[:,0]
Y=list(Y)

pred=[]
for iii, y in enumerate(Y):
    pred.append([aaa[iii], y+filtered_rank_dic[aaa[iii]]])

# sort by score from high to low
pred.sort(key=lambda x : x[1], reverse=True)
pred=np.array(pred)
pred=pred[:,0]
pred=list(pred)

for i in range(len(pred)):
    pred[i]=int(pred[i])

tot=0
for i in range(len(pred)):
    if pred[i]!=aaa[i]:
        tot+=1
tot

1007

In [14]:
user=len(hybrid_users)-1

aaa=list(filtered_rank_dic.keys())

# map movie ids to movie vocabulary number
X_bert=np.array(pd.Series(np.array(aaa)).map(bert_dic).tolist())
# map user ids to user vocabulary number
X_user=np.array([user for i in range(X_bert.shape[0])])

Y=hybrid_model.predict([X_user, pd.Series(aaa).map(filtered_movie_id_mapping).values, X_bert])
Y=Y[:,0]
Y=list(Y)

pred=[]
for iii, y in enumerate(Y):
    pred.append([aaa[iii], y+filtered_rank_dic[aaa[iii]]])

# sort by score from high to low
pred.sort(key=lambda x : x[1], reverse=True)
pred=np.array(pred)
pred=pred[:,0]
pred=list(pred)

for i in range(len(pred)):
    pred[i]=int(pred[i])

tot=0
for i in range(len(pred)):
    if pred[i]!=aaa[i]:
        tot+=1
tot

4231

**as we see from the result above, for a user who hasn't rated any movie, our movie recommendation largely align with the rank_dic. The difference is assumed to be due to the fact that although we have set L2 regularization terms for the user embedding, the actual user embeddings for the last 100 users are still not perfectly zero.**

**now retrain the model with a few arbitrary user-movie ratings**

In [15]:
# len(hybrid_movies)

In [16]:
hybrid_model = tf.keras.models.load_model('./hybrid.h5')
layers=hybrid_model.layers
layers[3].trainable=False
layers[9].trainable=False
layers[13].trainable=False
layers[14].trainable=False
layers[15].trainable=False

In [17]:
user=len(hybrid_users)+0

aaa=[7, 7**2, 7**3, 7**4, 100, 103]
for i in range(len(aaa)):
    aaa[i]=hybrid_movies[aaa[i]]
    
bbb=np.array([3,5,2,1,5,4])

# map movie ids to movie vocabulary number
X_bert=np.array(pd.Series(np.array(aaa)).map(bert_dic).tolist())
# map user ids to user vocabulary number
X_user=np.array([user for i in range(X_bert.shape[0])])

# 3 epochs because the model was trained with 3 epochs during the training phase
hybrid_model.fit([X_user, pd.Series(aaa).map(filtered_movie_id_mapping).values, X_bert], bbb, epochs=3)

Epoch 1/3
1/1 [==============================] - 1s 1s/step - loss: 15.3202
Epoch 2/3
1/1 [==============================] - 0s 270ms/step - loss: 15.1087
Epoch 3/3
1/1 [==============================] - 0s 244ms/step - loss: 14.8241


**make a prediciton again for one of the last 100 users, we see that this time the recommendation is vastly different from the rank_dic**

In [18]:
user=len(hybrid_users)+0

aaa=list(filtered_rank_dic.keys())

# map movie ids to movie vocabulary number
X_bert=np.array(pd.Series(np.array(aaa)).map(bert_dic).tolist())
# map user ids to user vocabulary number
X_user=np.array([user for i in range(X_bert.shape[0])])

Y=hybrid_model.predict([X_user, pd.Series(aaa).map(filtered_movie_id_mapping).values\
                 ,X_bert\
                ])
Y=Y[:,0]
Y=list(Y)

pred=[]
for iii, y in enumerate(Y):
    pred.append([aaa[iii], y+filtered_rank_dic[aaa[iii]]])

# sort by score from high to low
pred.sort(key=lambda x : x[1], reverse=True)
pred=np.array(pred)
pred=pred[:,0]
pred=list(pred)

for i in range(len(pred)):
    pred[i]=int(pred[i])

tot=0
for i in range(len(pred)):
    if pred[i]!=aaa[i]:
        tot+=1
tot

4173

**for users that are already in the training set, let's make predictions**

In [19]:
user=len(hybrid_users)-1

aaa=list(filtered_rank_dic.keys())

# map movie ids to movie vocabulary number
X_bert=np.array(pd.Series(np.array(aaa)).map(bert_dic).tolist())
# map user ids to user vocabulary number
X_user=np.array([user for i in range(X_bert.shape[0])])

Y=hybrid_model.predict([X_user, pd.Series(aaa).map(filtered_movie_id_mapping).values, X_bert])
Y=Y[:,0]
Y=list(Y)

pred=[]
for iii, y in enumerate(Y):
    pred.append([aaa[iii], y+filtered_rank_dic[aaa[iii]]])

# sort by score from high to low
pred.sort(key=lambda x : x[1], reverse=True)
pred=np.array(pred)
pred=pred[:,0]
pred=list(pred)

for i in range(len(pred)):
    pred[i]=int(pred[i])

tot=0
for i in range(len(pred)):
    if pred[i]!=aaa[i]:
        tot+=1
tot

4230

**notice the result is slightly different from the one above because of the regularization of the user embeddings**

In [20]:
with open('movie_id_vs_movie_title_list.txt', 'rb') as fp:
    movie_id_vs_movie_title_list=pickle.load(fp)

movie_id_title_dic={}
for ele in movie_id_vs_movie_title_list:
    movie_id_title_dic[ele[0]]=ele[1]
del movie_id_vs_movie_title_list

In [25]:
a=10
a += 1

In [26]:
a

11

In [27]:
5 >= 5

True

In [ ]:
rank_dic_copy = filtered_rank_dic.copy()

# list of movies already rated in train set
already = filtered_train_movie_id_dic[user]

# remove movies that are already in the train set
for ele in already:
    rank_dic_copy.pop(ele)

# save a copy of movie ids
aaa = list(rank_dic_copy.keys())

# map movie ids to movie vocabulary number
X_bert = np.array(pd.Series(np.array(aaa)).map(bert_dic).tolist())
# map user ids to user vocabulary number
X_user = np.array([user for i in range(X_bert.shape[0])])

Y=model.predict([X_user, pd.Series(aaa).map(movie_id_mapping).values, X_bert])
Y=Y[:,0]
Y=list(Y)

pred=[]
for iii, y in enumerate(Y):
    pred.append([aaa[iii], y+filtered_rank_dic[aaa[iii]]])

# sort by score from high to low
pred.sort(key=lambda x : x[1], reverse=True)
pred=np.array(pred)
pred=pred[:10,0]
predictions=pd.Series(pred).map(movie_id_title_dic).tolist()

result1.append(average_precision_at_k(rel, pred, 100000))

result2.append(average_precision_at_k(rel, aaa, 100000))

In [32]:
aaa=pd.Series(np.array([1,2,3])).map({1:'a',2:'b', 3:'c'}).tolist()
aaa

['a', 'b', 'c']

In [4]:
import pickle
filtered_train_movie_id_dic={}
with open('filtered_train_movie_ids_list.txt', 'rb') as fp:
    filtered_train_movie_ids_list = pickle.load(fp)
    for ele in filtered_train_movie_ids_list:
        filtered_train_movie_id_dic[ele[0]] = ele[1]

In [6]:
reversed_filtered_user_id_mapping_dic={}
with open('filtered_user_id_mapping_list.txt', 'rb') as fp:
    filtered_user_id_mapping_list = pickle.load(fp)
    for ele in filtered_user_id_mapping_list:
        reversed_filtered_user_id_mapping_dic[ele[1]] = ele[0]

In [9]:
reversed_filtered_user_id_mapping_dic[1]

'1437978'

In [22]:
len(reversed_filtered_user_id_mapping_dic)

150245

In [23]:
len(filtered_train_movie_id_dic)

57069

In [21]:
filtered_train_movie_id_dic[reversed_filtered_user_id_mapping_dic[18]]

[2680,
 8954,
 14545,
 16922,
 10170,
 6206,
 15532,
 14646,
 8467,
 3825,
 2782,
 15151,
 4590,
 9378,
 2495,
 12238,
 4801,
 14312,
 5897,
 3350,
 10024,
 12470,
 11398,
 7544,
 15078,
 3638,
 5997,
 16384,
 13458,
 16181,
 14109,
 8989,
 5434,
 15105,
 14970,
 9645,
 12034,
 4520,
 8437,
 15674,
 5318,
 4043,
 11677,
 14527,
 12355,
 7483,
 9442,
 8372,
 10240,
 15285,
 5695,
 1138,
 5317,
 1558,
 3085,
 3466,
 1661,
 12918,
 8387,
 118,
 14454,
 2254,
 16641,
 10607,
 16825,
 2391,
 4159,
 305,
 11337,
 2376,
 7745,
 17703,
 16552,
 8086,
 15473,
 6972,
 3475,
 13993,
 10359,
 16879,
 5472,
 4829,
 4951,
 12207,
 17053,
 12542,
 6119,
 12945,
 9156,
 1854,
 9756,
 17079,
 6042,
 2139,
 7617,
 14480,
 12047,
 1220,
 4380,
 9049,
 6350,
 658,
 11929,
 14313,
 2400,
 4583,
 16930,
 15063,
 6609,
 9780,
 4127,
 12570,
 5345,
 17513,
 2612,
 12934,
 11139,
 16438,
 13923,
 15048,
 2675,
 11254,
 7381,
 13440,
 14457,
 6760,
 15755,
 6386,
 13002,
 16912,
 13728,
 16698,
 9768,
 13748,
 

In [ ]:
import pandas as pd
df_train_filtered=pd.read_pickle('df_train_filtered.pkl')